In [1]:
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121 --only-binary :all:
!pip install -q transformers==4.38.2 accelerate==0.28.0 pandas scikit-learn wandb unidecode nltk seaborn tqdm emoji huggingface_hub matplotlib joblib xgboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.5/780.5 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 70.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 87.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
import re
import random
import emoji
import nltk
import torch
import warnings
import numpy as np
import pandas as pd
import torch.nn as nn
import matplotlib.pyplot as plt
import seaborn as sns
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm # Use tqdm.notebook for better notebook integration
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from nltk.tokenize import word_tokenize
from huggingface_hub import login
import joblib # For saving models
from kaggle_secrets import UserSecretsClient
from IPython.display import display # Import display for DataFrames

In [3]:
try:
    user_secrets = UserSecretsClient()
    HF_TOKEN = user_secrets.get_secret("HF_TOKEN")
    # WANDB_API_KEY = user_secrets.get_secret("WANDB_API_KEY") # Keep commented if not used
    print("Kaggle secrets loaded successfully.")
except Exception as e:
    print(f"Warning: Could not load Kaggle secrets. Using placeholder token. Error: {e}")
    HF_TOKEN = "YOUR_HF_TOKEN_PLACEHOLDER" # Fallback or error


try:
    nltk.download('punkt', quiet=True)
except Exception as e:
    print(f"Could not download nltk punkt: {e}")

warnings.filterwarnings("ignore")

try:
    login(token=HF_TOKEN)
except Exception as e:
    print(f"Hugging Face login failed (running locally?): {e}")
# import wandb # Not using W&B in this version

# Check CUDA availability and count GPUs
if torch.cuda.is_available():
    DEVICE = "cuda" # Still useful to know CUDA is the base device type
    num_gpus = torch.cuda.device_count()
    print(f"CUDA is available. Using device type: {DEVICE}")
    print(f"Number of GPUs available: {num_gpus}")
    if num_gpus < 2:
        print("Warning: Only one GPU detected. Multi-GPU usage is not automatically configured, but DataParallel wrapper will handle single GPU.")
    # Optionally set the primary device if needed later
    # torch.cuda.set_device(0) # Often device 0 is the default
else:
    DEVICE = "cpu"
    num_gpus = 0
    print(f"Using device: {DEVICE}")
    print("CUDA not available, multi-GPU usage is not possible.")

Kaggle secrets loaded successfully.
CUDA is available. Using device type: cuda
Number of GPUs available: 2


In [4]:
# Cell 4: Configuration
# Updated from AdaBoost to XGBoost configuration
class CFG:
    MODEL_NAME = "vinai/bertweet-base"
    MAX_LEN = 96
    BATCH_SIZE = 64
    NUM_WORKERS = 2
    SEED = 42
    DROPOUT = 0.2

    TRAIN_CSV = "/kaggle/input/twitter-emotion-dataset2/train.csv"
    VAL_CSV = "/kaggle/input/twitter-emotion-dataset2/val.csv"
    TEST_CSV = "/kaggle/input/twitter-emotion-dataset2/test.csv"

    BERT_MODEL_FULL_PATH = "/kaggle/input/bert-fine-tuned/pytorch/default/1/bert_fine_tune.pt"

    XGBOOST_SAVE_DIR = "/kaggle/working/"
    XGBOOST_MODEL_FILENAME = "xgboost_stacking_model.pkl" # Specific filename for XGBoost
    
    # Parameters provided by user
    XGB_PARAMS = {
        'colsample_bytree': 0.7,
        'learning_rate': 0.05,
        'max_depth': 3,
        'n_estimators': 100,
        'subsample': 0.7
    }

    FP16 = True if DEVICE == "cuda" else False # Enable FP16 only if CUDA is available

print("Configuration defined using Kaggle paths and XGBoost parameters.")
os.makedirs(CFG.XGBOOST_SAVE_DIR, exist_ok=True) # Updated path

Configuration defined using Kaggle paths and XGBoost parameters.


In [5]:
# Seeding and Preprocessing
def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

seed_everything(CFG.SEED)

def pre_process_document(text):
    if not isinstance(text, str):
        return ""
    text = re.sub(r"http\\S+|www\\S+|https\\S+", '', text)
    text = emoji.demojize(text)
    text = re.sub(r"@\\w+", "@user", text)
    text = re.sub(r"#(\\w+)", r"\\1", text)
    text = re.sub(r"\\s+", " ", text).strip()
    return text.lower()

print("Seeding and preprocessing function defined.")

Seeding and preprocessing function defined.


In [6]:
# Load Data, Preprocess, Encode Labels
print("\n--- Loading and Preprocessing Data ---")
try:
    train_df = pd.read_csv(CFG.TRAIN_CSV)
    val_df   = pd.read_csv(CFG.VAL_CSV)
    test_df  = pd.read_csv(CFG.TEST_CSV)
    print("DataFrames loaded successfully.")
except FileNotFoundError as e:
    print(f"Error loading data files: {e}. Please check the paths in CFG.")
    # Exit or handle error appropriately in a real script
    # For Kaggle notebook, we might continue with empty DFs if needed for structure
    train_df = pd.DataFrame(columns=['text', 'label'])
    val_df = pd.DataFrame(columns=['text', 'label'])
    test_df = pd.DataFrame(columns=['text', 'label'])


if not train_df.empty:
    print("Preprocessing text data...")
    train_df['text'] = train_df['text'].apply(pre_process_document)
    val_df['text']   = val_df['text'].apply(pre_process_document)
    test_df['text']  = test_df['text'].apply(pre_process_document)

    print("Encoding labels...")
    le = LabelEncoder()
    train_df['label'] = le.fit_transform(train_df['label'])
    try:
        val_df['label'] = le.transform(val_df['label'])
        test_df['label'] = le.transform(test_df['label'])
    except ValueError as e:
        print(f"Warning: {e}. Check if val/test sets contain labels not present in the training set.")

    NUM_CLASSES = len(le.classes_)
    CLASS_NAMES = list(le.classes_)
    print(f"Detected {NUM_CLASSES} emotion classes: {CLASS_NAMES}")
    print("Label encoding complete.")

    y_train = train_df['label'].values
    y_val = val_df['label'].values
    y_test = test_df['label'].values
else:
    print("DataFrames are empty, skipping preprocessing and label encoding.")
    le = None # Ensure le is defined even if empty
    NUM_CLASSES = 0
    CLASS_NAMES = []
    y_train, y_val, y_test = None, None, None


--- Loading and Preprocessing Data ---
DataFrames loaded successfully.
Preprocessing text data...
Encoding labels...
Detected 6 emotion classes: [0, 1, 2, 3, 4, 5]
Label encoding complete.


In [7]:
# Load Tokenizer
print(f"Loading tokenizer: {CFG.MODEL_NAME}")
tokenizer = AutoTokenizer.from_pretrained(CFG.MODEL_NAME, use_fast=True)

Loading tokenizer: vinai/bertweet-base


config.json:   0%|          | 0.00/558 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

bpe.codes: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [8]:
# Dataset Class
class TweetDataset(Dataset):
    def __init__(self, df, tokenizer, max_len):
        self.texts = df['text'].values
        self.labels = df['label'].astype(int).values if 'label' in df else None
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        enc = self.tokenizer(
            text,
            add_special_tokens=True,
            truncation=True,
            padding='max_length',
            max_length=self.max_len,
            return_tensors='pt'
        )
        item = {
            'input_ids': enc['input_ids'].squeeze(0),
            'attention_mask': enc['attention_mask'].squeeze(0)
        }
        if self.labels is not None:
            item['label'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

print("TweetDataset class defined.")

TweetDataset class defined.


In [9]:
# Create DataLoaders
print("\n--- Creating Datasets and DataLoaders for Feature Extraction ---")
if not train_df.empty:
    train_ds = TweetDataset(train_df, tokenizer, CFG.MAX_LEN)
    val_ds   = TweetDataset(val_df, tokenizer, CFG.MAX_LEN)
    test_ds  = TweetDataset(test_df, tokenizer, CFG.MAX_LEN)
    print("PyTorch Datasets created.")

    train_loader = DataLoader(train_ds, batch_size=CFG.BATCH_SIZE, shuffle=False, num_workers=CFG.NUM_WORKERS, pin_memory=(DEVICE=='cuda'))
    val_loader   = DataLoader(val_ds,   batch_size=CFG.BATCH_SIZE, shuffle=False, num_workers=CFG.NUM_WORKERS, pin_memory=(DEVICE=='cuda'))
    test_loader  = DataLoader(test_ds,  batch_size=CFG.BATCH_SIZE, shuffle=False, num_workers=CFG.NUM_WORKERS, pin_memory=(DEVICE=='cuda'))
    print("DataLoaders for feature extraction created.")
else:
    print("Skipping Dataset and DataLoader creation as DataFrames are empty.")
    train_ds, val_ds, test_ds = None, None, None
    train_loader, val_loader, test_loader = None, None, None


--- Creating Datasets and DataLoaders for Feature Extraction ---
PyTorch Datasets created.
DataLoaders for feature extraction created.


In [10]:
# BERTweet Classifier Class
class BERTweetClassifier(nn.Module):
    def __init__(self, num_labels):
        super().__init__()
        self.bert = AutoModel.from_pretrained(CFG.MODEL_NAME)
        self.dropout = nn.Dropout(CFG.DROPOUT)
        self.fc = nn.Linear(self.bert.config.hidden_size, num_labels)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden = outputs.last_hidden_state
        # Use mean pooling of the last hidden state
        mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden.size()).float()
        sum_hidden = torch.sum(last_hidden * mask_expanded, 1)
        sum_mask = mask_expanded.sum(1)
        sum_mask = torch.clamp(sum_mask, min=1e-9) # Avoid division by zero
        pooled = sum_hidden / sum_mask
        pooled = self.dropout(pooled)
        logits = self.fc(pooled)
        return logits

    def get_features(self, input_ids, attention_mask):
         outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
         last_hidden = outputs.last_hidden_state
         # Use mean pooling of the last hidden state
         mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden.size()).float()
         sum_hidden = torch.sum(last_hidden * mask_expanded, 1)
         sum_mask = mask_expanded.sum(1)
         sum_mask = torch.clamp(sum_mask, min=1e-9) # Avoid division by zero
         pooled = sum_hidden / sum_mask
         return pooled # Return the pooled features before dropout/fc

print("BERTweetClassifier class defined with get_features method.")

BERTweetClassifier class defined with get_features method.


In [11]:
# Load Pre-trained BERT Model & Verification
print(f"\n--- Loading Pre-trained BERT Model ---")
bert_model_full_path = CFG.BERT_MODEL_FULL_PATH
print(f"Attempting to load model from: {bert_model_full_path}")

bert_model_for_features = None
bert_test_preds = None # To store predictions from BERT model for comparison later if needed
# bert_test_labels = y_test # y_test might not be defined if data loading failed

# Check if the directory exists, create if not (relevant if running locally)
if not os.path.exists(os.path.dirname(CFG.BERT_MODEL_FULL_PATH)):
     os.makedirs(os.path.dirname(CFG.BERT_MODEL_FULL_PATH), exist_ok=True)
     print(f"Created directory: {os.path.dirname(CFG.BERT_MODEL_FULL_PATH)}")
     print(f"Please place the pre-trained model file '{os.path.basename(CFG.BERT_MODEL_FULL_PATH)}' in this directory.")

try:
    if not os.path.exists(bert_model_full_path):
         raise FileNotFoundError(f"Model file not found at {bert_model_full_path}. Please check the path.")

    state_dict = torch.load(bert_model_full_path, map_location='cpu')
    # Check if the state_dict keys start with 'module.' (indicating DataParallel save)
    is_data_parallel = list(state_dict.keys())[0].startswith('module.')

    # Instantiate the model structure FIRST
    model_instance = BERTweetClassifier(NUM_CLASSES) # Make sure NUM_CLASSES is defined

    # If saved with DataParallel, remove the 'module.' prefix
    if is_data_parallel:
        print("Detected 'module.' prefix in state_dict, removing it for loading.")
        # Create a new state dictionary without the prefix
        state_dict = {k[len('module.'):]: v for k, v in state_dict.items()}

    # Load the state dict into the model instance
    model_instance.load_state_dict(state_dict)

    # Apply DataParallel if multiple GPUs are available AFTER loading state dict
    if num_gpus > 1 and DEVICE == "cuda":
        print(f"Using {num_gpus} GPUs with nn.DataParallel.")
        bert_model_for_features = nn.DataParallel(model_instance)
        bert_model_for_features.to(DEVICE) # Move the DataParallel wrapper to device
    else:
        bert_model_for_features = model_instance.to(DEVICE) # Move the single model instance

    bert_model_for_features.eval() # Set to evaluation mode
    print(f"Pre-trained BERT model loaded successfully from {bert_model_full_path}")

    # --- Verification Step ---
    if test_loader and y_test is not None: # Ensure loader and labels exist
        print("\nVerifying loaded BERT model on the test set...")
        @torch.no_grad()
        def verify_bert_model(model, loader, device):
            # Get the actual model if wrapped in DataParallel
            actual_model = model.module if isinstance(model, nn.DataParallel) else model
            actual_model.eval() # Ensure model is in eval mode

            all_preds_verify, all_labels_verify = [], []
            criterion_verify = nn.CrossEntropyLoss()
            total_loss_verify = 0

            for batch in tqdm(loader, desc="BERT Verifying", leave=False):
                input_ids = batch['input_ids'].to(device, non_blocking=True)
                attention_mask = batch['attention_mask'].to(device, non_blocking=True)
                labels = batch.get('label') # Use .get() for safety
                if labels is not None:
                    labels = labels.to(device, non_blocking=True)

                # Use autocast for potential FP16 inference
                with torch.cuda.amp.autocast(enabled=CFG.FP16):
                     outputs = model(input_ids, attention_mask) # Use the potentially wrapped model
                     if labels is not None:
                         loss = criterion_verify(outputs, labels)
                         total_loss_verify += loss.item() * input_ids.size(0) # Accumulate total loss

                batch_preds = outputs.argmax(dim=-1).cpu().numpy()
                all_preds_verify.extend(batch_preds)
                if labels is not None:
                    batch_labels = labels.cpu().numpy()
                    all_labels_verify.extend(batch_labels)

            avg_loss = total_loss_verify / len(loader.dataset) if loader.dataset else 0
            acc, f1 = None, None
            if all_labels_verify: # Only calculate metrics if labels were present
                acc = accuracy_score(all_labels_verify, all_preds_verify)
                f1 = f1_score(all_labels_verify, all_preds_verify, average='weighted')
            # Return predictions even if labels are missing
            return avg_loss, acc, f1, np.array(all_preds_verify), np.array(all_labels_verify) if all_labels_verify else None


        test_loss, test_acc, test_f1, bert_test_preds_verify, _ = verify_bert_model(
            bert_model_for_features, test_loader, DEVICE
        )

        if test_acc is not None and test_f1 is not None:
            print(f"\n✅ Loaded BERT Model Test Results (Verification):")
            print(f"   Test Loss: {test_loss:.4f}")
            print(f"   Test Acc:  {test_acc:.4f}")
            print(f"   Test F1:   {test_f1:.4f}")
            bert_test_preds = bert_test_preds_verify # Store verified predictions
        else:
            print("   Verification metrics could not be calculated (likely missing test labels).")
    else:
        print("\nSkipping BERT model verification (test data or labels not available).")


except FileNotFoundError as e:
    print(f"Error: {e}")
    bert_model_for_features = None # Ensure model is None if loading failed
except Exception as e:
    print(f"An error occurred loading the pre-trained BERT model: {e}")
    bert_model_for_features = None # Ensure model is None if loading failed

if bert_model_for_features is None:
    print("❌ BERT model loading failed. Cannot proceed with feature extraction.")
else:
     print("✅ BERT model ready for feature extraction.")


--- Loading Pre-trained BERT Model ---
Attempting to load model from: /kaggle/input/bert-fine-tuned/pytorch/default/1/bert_fine_tune.pt


pytorch_model.bin:   0%|          | 0.00/543M [00:00<?, ?B/s]

Using 2 GPUs with nn.DataParallel.
Pre-trained BERT model loaded successfully from /kaggle/input/bert-fine-tuned/pytorch/default/1/bert_fine_tune.pt

Verifying loaded BERT model on the test set...


BERT Verifying:   0%|          | 0/233 [00:00<?, ?it/s]


✅ Loaded BERT Model Test Results (Verification):
   Test Loss: 0.1547
   Test Acc:  0.9702
   Test F1:   0.9701
✅ BERT model ready for feature extraction.


In [12]:
# Feature Collection Function
@torch.no_grad()
def collect_features(model, loader, device):
    # Get the actual model if wrapped in DataParallel
    actual_model = model.module if isinstance(model, nn.DataParallel) else model
    actual_model.eval() # Ensure model is in eval mode

    all_features = []
    print(f"Collecting features using device: {device}")
    for batch in tqdm(loader, desc=f"Collecting Features"):
        input_ids = batch['input_ids'].to(device, non_blocking=True)
        attention_mask = batch['attention_mask'].to(device, non_blocking=True)

        # Use autocast only if FP16 is enabled and on CUDA
        with torch.cuda.amp.autocast(enabled=(CFG.FP16 and device=='cuda')):
            # Call the get_features method of the actual model
            features = actual_model.get_features(input_ids, attention_mask)

        all_features.append(features.cpu().numpy()) # Move features to CPU before converting to numpy

    if not all_features:
        raise ValueError("No features were collected. The loader might be empty.")

    # Concatenate features from all batches
    return np.concatenate(all_features, axis=0)

print("Feature collection function defined.")

Feature collection function defined.


In [13]:
# Extract Features
X_train_features, X_val_features, X_test_features = None, None, None

if bert_model_for_features and train_loader: # Check if model and loaders exist
    print(f"\n--- Collecting Feature Vectors from BERT Model ---")
    try:
        X_train_features = collect_features(bert_model_for_features, train_loader, DEVICE)
        X_val_features   = collect_features(bert_model_for_features, val_loader, DEVICE)
        X_test_features  = collect_features(bert_model_for_features, test_loader, DEVICE)

        # Assertion checks (use actual model for config)
        actual_model = bert_model_for_features.module if isinstance(bert_model_for_features, nn.DataParallel) else bert_model_for_features
        bert_hidden_size = actual_model.bert.config.hidden_size # Get hidden size from the underlying BERT model

        assert X_train_features.shape == (len(train_ds), bert_hidden_size), f"Train feature shape mismatch: Expected {(len(train_ds), bert_hidden_size)}, Got {X_train_features.shape}"
        assert X_val_features.shape == (len(val_ds), bert_hidden_size), f"Validation feature shape mismatch: Expected {(len(val_ds), bert_hidden_size)}, Got {X_val_features.shape}"
        assert X_test_features.shape == (len(test_ds), bert_hidden_size), f"Test feature shape mismatch: Expected {(len(test_ds), bert_hidden_size)}, Got {X_test_features.shape}"

        print("✅ Done collecting feature vectors.")
        print(f"Feature Shapes: Train {X_train_features.shape}, Val {X_val_features.shape}, Test {X_test_features.shape}")

    except ValueError as e:
        print(f"Error during feature collection: {e}")
        X_train_features, X_val_features, X_test_features = None, None, None
    except Exception as e:
        print(f"An unexpected error occurred during feature collection: {e}")
        X_train_features, X_val_features, X_test_features = None, None, None

else:
    print("\nSkipping feature extraction as the BERT model or data loaders were not available.")


--- Collecting Feature Vectors from BERT Model ---


✅ Done collecting feature vectors.
Feature Shapes: Train (52109, 768), Val (7445, 768), Test (14888, 768)


In [14]:
# Cell 14: Scale Features, Train XGBoost Stacker
best_xgb_on_features = None
scaler = None
xgb_train_preds, xgb_val_preds, xgb_test_preds = None, None, None # Initialize prediction variables

if X_train_features is not None and y_train is not None:
    print("\n--- Scaling Features and Training XGBoost Stacker ---")

    # 1. Use sklearn.preprocessing.StandardScaler (same as before)
    scaler = StandardScaler()
    print("Fitting Scaler and Transforming Train Features...")
    X_train_scaled = scaler.fit_transform(X_train_features)
    print("Transforming Validation Features...")
    X_val_scaled   = scaler.transform(X_val_features)
    print("Transforming Test Features...")
    X_test_scaled  = scaler.transform(X_test_features)
    print("Feature scaling complete.")

    print(f"\n--- Training XGBoost on Features with provided parameters ---")
    print(f"Params: {CFG.XGB_PARAMS}")

    # 2. Initialize XGBClassifier with specific parameters from CFG
    best_xgb_on_features = xgb.XGBClassifier(
        **CFG.XGB_PARAMS, # Unpack the parameters
        objective='multi:softmax',
        num_class=NUM_CLASSES,
        random_state=CFG.SEED,
        eval_metric='mlogloss',
        use_label_encoder=False # To avoid warning
    )

    # 3. Fit the XGBoost model
    print("Fitting XGBoost model...")
    best_xgb_on_features.fit(X_train_scaled, y_train)
    print("XGBoost fitting complete.")

    # --- Evaluation --
    print("\n--- Evaluating Trained XGBoost Stacker ---")
    def calculate_metrics(labels, preds):
        return {
            "accuracy": accuracy_score(labels, preds),
            "f1": f1_score(labels, preds, average='weighted'),
            "precision": precision_score(labels, preds, average='weighted', zero_division=0),
            "recall": recall_score(labels, preds, average='weighted', zero_division=0)
        }

    def eval_and_return(model, X, y, desc="Evaluating"):
        print(f"Predicting on {desc} set...")
        preds = model.predict(X)
        print("Calculating metrics...")
        metrics = calculate_metrics(y, preds)
        return metrics, preds # Return predictions as well

    # 4. Evaluate the trained XGBoost model
    xgb_train_metrics, xgb_train_preds = eval_and_return(best_xgb_on_features, X_train_scaled, y_train, desc="Train")
    xgb_val_metrics, xgb_val_preds     = eval_and_return(best_xgb_on_features, X_val_scaled, y_val, desc="Validation")
    xgb_test_metrics, xgb_test_preds   = eval_and_return(best_xgb_on_features, X_test_scaled, y_test, desc="Test")

    xgb_results_feat = pd.DataFrame({
        'Train': xgb_train_metrics,
        'Validation': xgb_val_metrics,
        'Test': xgb_test_metrics
    }).T

    print("\n📊 Trained XGBoost Stacking Performance 📊")
    display(xgb_results_feat)

else:
    print("\nXGBoost training skipped as features or labels were not available.")


--- Scaling Features and Training XGBoost Stacker ---
Fitting Scaler and Transforming Train Features...
Transforming Validation Features...
Transforming Test Features...
Feature scaling complete.

--- Training XGBoost on Features with provided parameters ---
Params: {'colsample_bytree': 0.7, 'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.7}
Fitting XGBoost model...
XGBoost fitting complete.

--- Evaluating Trained XGBoost Stacker ---
Predicting on Train set...
Calculating metrics...
Predicting on Validation set...
Calculating metrics...
Predicting on Test set...
Calculating metrics...

📊 Trained XGBoost Stacking Performance 📊


,accuracy,f1,precision,recall
Train,0.983765,0.983766,0.984128,0.983765
Validation,0.968570,0.968504,0.968987,0.968570
Test,0.968364,0.968309,0.968684,0.968364


In [15]:
# Cell 15: Save XGBoost Model, Scaler, Label Encoder

if best_xgb_on_features and scaler and le:
    print("\n--- Saving XGBoost Model, Scaler, and Label Encoder ---")
    os.makedirs(CFG.XGBOOST_SAVE_DIR, exist_ok=True) # Ensure directory exists

    xgboost_save_path = os.path.join(CFG.XGBOOST_SAVE_DIR, CFG.XGBOOST_MODEL_FILENAME)
    try:
        # Save the XGBoost model, the scaler, and the label encoder together
        joblib.dump((best_xgb_on_features, scaler, le), xgboost_save_path)
        print(f"\n✅ Saved Trained XGBoost (on features) + Scaler + Label Encoder to: {xgboost_save_path}")
    except Exception as e:
        print(f"❌ Error saving XGBoost model artifacts: {e}")
else:
    print("\nSkipping saving of XGBoost artifacts as the model, scaler, or encoder is not available.")


--- Saving XGBoost Model, Scaler, and Label Encoder ---

✅ Saved Trained XGBoost (on features) + Scaler + Label Encoder to: /kaggle/working/xgboost_stacking_model.pkl


In [16]:
# Cell 16: Confusion Matrix Plotting for XGBoost

if xgb_train_preds is not None and y_train is not None: # Check if predictions exist
    print("\n--- Generating Confusion Matrices for Trained XGBoost on Features ---")
    def plot_confusion_matrix(labels, preds, class_names, title, filename="confusion_matrix.png"):
        try:
            # Calculate CM without normalization first for raw counts if desired,
            # but normalize for heatmap visualization consistency
            cm = confusion_matrix(labels, preds, normalize='true') # Normalize
            plt.figure(figsize=(8, 6)) # Create a new figure for each plot
            sns.heatmap(cm, annot=True, fmt='.2f', cmap='Blues',
                        xticklabels=class_names, yticklabels=class_names)
            plt.title(title)
            plt.xlabel('Predicted Label')
            plt.ylabel('True Label')
            plt.tight_layout()
            full_filename = os.path.join(CFG.XGBOOST_SAVE_DIR, filename) # Use the correct save directory
            plt.savefig(full_filename)
            print(f"Saved confusion matrix plot to: {full_filename}")
            # plt.show() # Prevent showing plot inline if saving is sufficient
            plt.close() # Close the figure to free memory
        except Exception as e:
            print(f"Error plotting confusion matrix '{title}': {e}")

    # Ensure CLASS_NAMES is available
    if CLASS_NAMES:
        plot_confusion_matrix(y_train, xgb_train_preds, CLASS_NAMES,
                              'Trained XGBoost (Features) Train CM (Normalized)',
                              filename="xgboost_train_cm.png") # Updated filename
        plot_confusion_matrix(y_val, xgb_val_preds, CLASS_NAMES,
                              'Trained XGBoost (Features) Validation CM (Normalized)',
                              filename="xgboost_val_cm.png") # Updated filename
        plot_confusion_matrix(y_test, xgb_test_preds, CLASS_NAMES,
                              'Trained XGBoost (Features) Test CM (Normalized)',
                              filename="xgboost_test_cm.png") # Updated filename
    else:
        print("Skipping confusion matrix plotting as class names are not defined.")
else:
    print("\nSkipping Confusion Matrix plotting as XGBoost predictions or labels are not available.")

print("\n--- Notebook Execution Complete ---")


--- Generating Confusion Matrices for Trained XGBoost on Features ---
Saved confusion matrix plot to: /kaggle/working/xgboost_train_cm.png
Saved confusion matrix plot to: /kaggle/working/xgboost_val_cm.png
Saved confusion matrix plot to: /kaggle/working/xgboost_test_cm.png

--- Notebook Execution Complete ---
